In [1]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

In [2]:
def get_face_reference(frame):
    (x, y, w, h) = face_cascade.detectMultiScale(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), 1.3, 5)[0]
    
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255, 0, 0), 2)

    face = frame[x:x+w][y:y+h]
    frame = np.subtract(frame[::], 200)
    
    return { 'pos_center': [x+w/2, y+h/2], 'area': w*h }

In [3]:
def compare_reference(reference, other_reference):
    dist_location = np.array(reference['pos_center']) - np.array(other_reference['pos_center'])
    dist_area = reference['area'] - other_reference['area']
    
    if reference['area'] + 10000 > other_reference['area']:
        move_area = {'move': 'zoom', 'dif': dist_area}
    elif reference['area'] - 10000 < other_reference['area']:
        move_area = {'move': 'unzoom', 'dif': dist_area}
    else: move_area = {'move': 'nothing', 'dif': dist_area}
    
    
    if dist_location[0] > 30:
        move_x = {'move': 'left', 'dif': dist_location[0]}
    elif dist_location[0] < -30:
        move_x = {'move': 'left', 'dif': dist_location[0]}
    else: move_x = {'move': 'right', 'dif': dist_location[0]}
    
    if dist_location[1] > 30:
        move_y = {'move': 'up', 'dif': dist_location[1]}
    elif dist_location[1] < -30:
        move_y = {'move': 'down', 'dif': dist_location[1]}
    else: move_y = {'move': 'nothing', 'dif': dist_location[1]}
    
    return [move_x, move_y, move_area]

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640);
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360);

ret, frame = cap.read()
reference = get_face_reference(frame)

while(True):
    try:
        ret, frame = cap.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        other_reference = get_face_reference(frame)

        actions = compare_reference(reference, other_reference)    

        text = ''
        for action in actions:
            text += action['move'] + ' | '

        cv2.putText(frame, text, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)

        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except:
        pass

In [ ]:
cap.release()
cv2.destroyAllWindows()